# Spacy Pre-Trained NER Deployment Example

[Container Structure](https://sagemaker-workshop.com/custom/containers.html)
- NER
    - predictor.py: (Flask app for inference)
    - wsgi.py: (Wrapper around predictor)
    - nginx.conf: (Config for nginx front-end)
    - serve: program for container hosting, launches gunicorn server
    - Note that there is no train for pre-trained
- Dockerfile

# Push Docker Image to ECR

In [5]:
%%sh

# Name of algo -> ECR
algorithm_name=sm-pretrained-spacy

cd container

#make serve executable
chmod +x NER/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Region, defaults to us-west-2
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  13.31kB
Step 1/10 : FROM python:3.8
 ---> 9fa000b720b7
Step 2/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> bf62ed56087d
Step 3/10 : RUN wget https://bootstrap.pypa.io/get-pip.py && python3 get-pip.py &&     pip install flask gevent gunicorn &&         rm -rf /root/.cache
 ---> Using cache
 ---> 6ecaf9bef126
Step 4/10 : RUN pip install spacy
 ---> Using cache
 ---> 6a717db7be19
Step 5/10 : RUN python -m spacy download en
 ---> Using cache
 ---> 08975d5a6c6d
Step 6/10 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 0b4f79503b30
Step 7/10 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> b79810c9f7fc
Step 8/10 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 0e505bb5c821
Step 9/10 : COPY NER /opt/program
 ---> Using cache
 ---> 2b90b6fd5438
Step 10/10 : WOR

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

EOF


CalledProcessError: Command 'b'\n# Name of algo -> ECR\nalgorithm_name=sm-pretrained-spacy\n\ncd container\n\n#make serve executable\nchmod +x NER/serve\n\naccount=$(aws sts get-caller-identity --query Account --output text)\n\n# Region, defaults to us-west-2\nregion=$(aws configure get region)\nregion=${region:-us-east-1}\n\nfullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"\n\n# If the repository doesn\'t exist in ECR, create it.\naws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1\n\nif [ $? -ne 0 ]\nthen\n    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null\nfi\n\n# Get the login command from ECR and execute it directly\naws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}\n\n# Build the docker image locally with the image name and then push it to ECR\n# with the full name.\n\ndocker build  -t ${algorithm_name} .\ndocker tag ${algorithm_name} ${

# SageMaker Client Setup

- [SageMaker Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model): Model, Endpoint Config, and Endpoint Creation
- [SageMaker RunTime Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint): Endpoint Invocation/Testing

In [7]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

#not really used in this use case, use when need to store model artifacts (Ex: MME)
s3_bucket = 'spacy-sagemaker-us-east-1-bucket'

role = get_execution_role()

# Model Creation
[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model)

In [8]:
from time import gmtime, strftime

model_name = 'spacy-nermodel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 's3://{}/spacy/'.format(s3_bucket) ## MODEL S3 URL
container = '{}.dkr.ecr.{}.amazonaws.com/sm-pretrained-spacy:latest'.format(account_id, region)
instance_type = 'ml.c5d.18xlarge'

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: spacy-nermodel-2022-09-19-21-16-25
Model data Url: s3://spacy-sagemaker-us-east-1-bucket/spacy/
Container image: 572320329544.dkr.ecr.us-east-1.amazonaws.com/sm-pretrained-spacy:latest
Model Arn: arn:aws:sagemaker:us-east-1:572320329544:model/spacy-nermodel-2022-09-19-21-16-25


# Endpoint Config Creation

[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config)

In [9]:
endpoint_config_name = 'spacy-ner-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: spacy-ner-config2022-09-19-21-16-28
Endpoint config Arn: arn:aws:sagemaker:us-east-1:572320329544:endpoint-config/spacy-ner-config2022-09-19-21-16-28


# Endpoint Creation
[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint)

In [10]:
%%time

import time

endpoint_name = 'spacy-ner-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: spacy-ner-endpoint2022-09-19-21-16-29
Endpoint Arn: arn:aws:sagemaker:us-east-1:572320329544:endpoint/spacy-ner-endpoint2022-09-19-21-16-29
Endpoint Status: Creating
Waiting for spacy-ner-endpoint2022-09-19-21-16-29 endpoint to be in service...
CPU times: user 60.1 ms, sys: 16.2 ms, total: 76.3 ms
Wall time: 2min


# Endpoint Invocation

[Documentation](https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint)

In [11]:
import json
content_type = "application/json"
request_body = {"input": "This is a test with NER in America with Amazon and Microsoft in Seattle, writing random stuff."}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    CustomAttributes="foobar",
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())['output']
result

[['NER', 'ORG'],
 ['America', 'GPE'],
 ['Amazon', 'ORG'],
 ['Microsoft', 'ORG'],
 ['Seattle', 'GPE']]

In [12]:
print(response)

{'ResponseMetadata': {'RequestId': '6352b781-e337-449f-b8e5-4d92125d5da0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6352b781-e337-449f-b8e5-4d92125d5da0', 'x-amzn-sagemaker-custom-attributes': 'fromServerfoobar', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Mon, 19 Sep 2022 21:18:30 GMT', 'content-type': 'application/json', 'content-length': '109'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'CustomAttributes': 'fromServerfoobar', 'Body': <botocore.response.StreamingBody object at 0x7f85b6722190>}
